In [ ]:
import pandas as pd
import requests
import zipfile
import io

print("Downloading MovieLens dataset...")

url = "https://files.grouplens.org/datasets/movielens/ml-latest-small.zip"
response = requests.get(url)
z = zipfile.ZipFile(io.BytesIO(response.content))
z.extractall()

print("Download complete!\n")

# Load the data
movies = pd.read_csv('ml-latest-small/movies.csv')
ratings = pd.read_csv('ml-latest-small/ratings.csv')
tags = pd.read_csv('ml-latest-small/tags.csv')
links = pd.read_csv('ml-latest-small/links.csv')

print("Movies:")
print(movies.head())
print(f"\nTotal movies: {len(movies)}")

print("\n\nRatings:")
print(ratings.head())
print(f"\nTotal ratings: {len(ratings)}")

print("\n\nTags:")
print(tags.head())
print(f"\nTotal tags: {len(tags)}")

print(f"\n\nDataset Summary:")
print(f"Movies: {len(movies)}")
print(f"Ratings: {len(ratings)}")
print(f"Users: {ratings['userId'].nunique()}")
print(f"Average rating: {ratings['rating'].mean():.2f}")

# Save for later use
movies.to_csv('movies_data.csv', index=False)
ratings.to_csv('ratings_data.csv', index=False)
tags.to_csv('tags_data.csv', index=False)

print("\n✅ Files saved!")

# Auto-download in Colab
try:
    from google.colab import files
    files.download('movies_data.csv')
    files.download('ratings_data.csv')
    files.download('tags_data.csv')
    print("Files downloaded to your computer!")
except:
    print("Files saved locally")

Download complete!

Movies:
   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  

Total movies: 9742


Ratings:
   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931

Total ratings: 100836


Tags:
   userId  movieId              tag   timestamp
0       2    60756            funny  14

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Files downloaded to your computer!


In [ ]:
import pandas as pd
import json
import numpy as np
from datetime import datetime

print("=" * 80)
print("DATA TRANSFORMATION FOR POLYGLOT DATABASES")
print("=" * 80)

# Load datasets
movies = pd.read_csv('movies_data.csv')
ratings = pd.read_csv('ratings_data.csv')
tags = pd.read_csv('tags_data.csv')

print(f"\nLoaded {len(movies)} movies, {len(ratings)} ratings, {len(tags)} tags\n")

# ============================================================================
# PostgreSQL - Rating Analytics
# ============================================================================
print("Processing PostgreSQL data...")

movie_stats = ratings.groupby('movieId').agg({
    'rating': ['count', 'mean', 'std'],
    'userId': 'nunique'
}).reset_index()

movie_stats.columns = ['movieId', 'num_ratings', 'avg_rating', 'rating_std', 'num_users']
movie_stats = movie_stats.merge(movies[['movieId', 'title', 'genres']], on='movieId')

# Rating distribution
rating_dist = ratings.groupby(['movieId', 'rating']).size().reset_index(name='count')

# Temporal analysis
ratings['timestamp'] = pd.to_datetime(ratings['timestamp'], unit='s')
ratings['year'] = ratings['timestamp'].dt.year
ratings['month'] = ratings['timestamp'].dt.month

temporal = ratings.groupby(['year', 'month']).agg({
    'rating': ['count', 'mean']
}).reset_index()
temporal.columns = ['year', 'month', 'num_ratings', 'avg_rating']

movie_stats.to_csv('postgres_movie_stats.csv', index=False)
rating_dist.to_csv('postgres_rating_distribution.csv', index=False)
temporal.to_csv('postgres_temporal_analysis.csv', index=False)

print(f"✓ Created 3 PostgreSQL tables")

# ============================================================================
# MongoDB - Movie Documents
# ============================================================================
print("Processing MongoDB data...")

mongo_docs = []

for _, movie in movies.iterrows():
    movie_ratings = ratings[ratings['movieId'] == movie['movieId']]
    movie_tags = tags[tags['movieId'] == movie['movieId']]

    doc = {
        'movieId': int(movie['movieId']),
        'title': movie['title'],
        'genres': movie['genres'].split('|') if pd.notna(movie['genres']) else [],
        'ratings': {
            'count': len(movie_ratings),
            'average': float(movie_ratings['rating'].mean()) if len(movie_ratings) > 0 else None,
            'distribution': movie_ratings['rating'].value_counts().to_dict() if len(movie_ratings) > 0 else {}
        },
        'tags': movie_tags['tag'].tolist() if len(movie_tags) > 0 else [],
        'metadata': {
            'year': movie['title'][-5:-1] if '(' in movie['title'] else None,
            'num_users': int(movie_ratings['userId'].nunique()) if len(movie_ratings) > 0 else 0
        }
    }
    mongo_docs.append(doc)

with open('mongodb_movies.json', 'w') as f:
    json.dump(mongo_docs, f, indent=2)

print(f"✓ Created {len(mongo_docs)} MongoDB documents")

# ============================================================================
# ChromaDB - Movie Descriptions for Recommendations
# ============================================================================
print("Processing ChromaDB data...")

chroma_docs = []

for _, movie in movies.iterrows():
    movie_tags = tags[tags['movieId'] == movie['movieId']]

    # Create text description
    genres = movie['genres'].replace('|', ', ') if pd.notna(movie['genres']) else ''
    tag_text = ', '.join(movie_tags['tag'].tolist()) if len(movie_tags) > 0 else ''

    description = f"{movie['title']}. Genres: {genres}. Tags: {tag_text}"

    chroma_docs.append({
        'id': f"movie_{int(movie['movieId'])}",
        'movieId': int(movie['movieId']),
        'title': movie['title'],
        'description': description,
        'genres': movie['genres'],
        'metadata': {
            'num_tags': len(movie_tags)
        }
    })

with open('chromadb_movies.json', 'w') as f:
    json.dump(chroma_docs, f, indent=2)

print(f"✓ Created {len(chroma_docs)} ChromaDB documents")

# ============================================================================
# Neo4j - User-Movie Graph
# ============================================================================
print("Processing Neo4j data...")

# User nodes
users = ratings[['userId']].drop_duplicates()
users['label'] = 'User'
users.to_csv('neo4j_users.csv', index=False)

# Movie nodes
movie_nodes = movies[['movieId', 'title', 'genres']].copy()
movie_nodes['label'] = 'Movie'
movie_nodes.to_csv('neo4j_movies.csv', index=False)

# Genre nodes
all_genres = set()
for g in movies['genres'].dropna():
    all_genres.update(g.split('|'))

genre_nodes = pd.DataFrame({'genre': list(all_genres), 'label': 'Genre'})
genre_nodes.to_csv('neo4j_genres.csv', index=False)

# Relationships: User -[RATED]-> Movie
user_movie = ratings[['userId', 'movieId', 'rating', 'timestamp']].copy()
user_movie['relationship'] = 'RATED'
user_movie.to_csv('neo4j_user_ratings.csv', index=False)

# Relationships: Movie -[HAS_GENRE]-> Genre
movie_genre = []
for _, movie in movies.iterrows():
    if pd.notna(movie['genres']):
        for genre in movie['genres'].split('|'):
            movie_genre.append({
                'movieId': movie['movieId'],
                'genre': genre,
                'relationship': 'HAS_GENRE'
            })

pd.DataFrame(movie_genre).to_csv('neo4j_movie_genres.csv', index=False)

print(f"✓ Created 3 node types and 2 relationship types")

# ============================================================================
# Summary
# ============================================================================
print("\n" + "=" * 80)
print("TRANSFORMATION COMPLETE")
print("=" * 80)
print("\nPostgreSQL:")
print("  - postgres_movie_stats.csv")
print("  - postgres_rating_distribution.csv")
print("  - postgres_temporal_analysis.csv")

print("\nMongoDB:")
print("  - mongodb_movies.json")

print("\nChromaDB:")
print("  - chromadb_movies.json")

print("\nNeo4j:")
print("  - neo4j_users.csv")
print("  - neo4j_movies.csv")
print("  - neo4j_genres.csv")
print("  - neo4j_user_ratings.csv")
print("  - neo4j_movie_genres.csv")

# Auto-download in Colab
try:
    from google.colab import files

    files.download('postgres_movie_stats.csv')
    files.download('postgres_rating_distribution.csv')
    files.download('postgres_temporal_analysis.csv')
    files.download('mongodb_movies.json')
    files.download('chromadb_movies.json')
    files.download('neo4j_users.csv')
    files.download('neo4j_movies.csv')
    files.download('neo4j_genres.csv')
    files.download('neo4j_user_ratings.csv')
    files.download('neo4j_movie_genres.csv')

    print("\n✓ All files downloaded")
except:
    print("\n✓ Files saved locally")

DATA TRANSFORMATION FOR POLYGLOT DATABASES

Loaded 9742 movies, 100836 ratings, 3683 tags

Processing PostgreSQL data...
✓ Created 3 PostgreSQL tables
Processing MongoDB data...
✓ Created 9742 MongoDB documents
Processing ChromaDB data...
✓ Created 9742 ChromaDB documents
Processing Neo4j data...
✓ Created 3 node types and 2 relationship types

TRANSFORMATION COMPLETE

PostgreSQL:
  - postgres_movie_stats.csv
  - postgres_rating_distribution.csv
  - postgres_temporal_analysis.csv

MongoDB:
  - mongodb_movies.json

ChromaDB:
  - chromadb_movies.json

Neo4j:
  - neo4j_users.csv
  - neo4j_movies.csv
  - neo4j_genres.csv
  - neo4j_user_ratings.csv
  - neo4j_movie_genres.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


✓ All files downloaded
